<a href="https://colab.research.google.com/github/martin-ss/Semantic-Segmentation-COVID-19/blob/main/Unet_basic_layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras
from tensorflow.keras import layers
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
input=keras.Input((572,572,3)) # this layer is in core layer of keras documentation
print(input.shape)

Using TensorFlow backend.


(None, 572, 572, 3)


In [ ]:
x=input
C1= tf.keras.layers.Conv2D(64, 3, activation='relu')(x)
print(C1.shape)

(None, 570, 570, 64)


In [ ]:
C2= tf.keras.layers.Conv2D(64, 3, activation='relu')(C1)
print(C2.shape)

(None, 568, 568, 64)


In [ ]:
Pool1=tf.keras.layers.MaxPool2D(pool_size=(2,2),padding='valid')(C2)
print(Pool1.shape)


(None, 284, 284, 64)


In [ ]:
P_C1=tf.keras.layers.Conv2D(128,3,activation='relu')(Pool1)
print(P_C1.shape)


(None, 282, 282, 128)


In [ ]:
P_C2=tf.keras.layers.Conv2D(128,3,activation='relu')(P_C1)
print(P_C2.shape)


(None, 280, 280, 128)


In [ ]:
P2=tf.keras.layers.MaxPool2D(pool_size=(2,2),padding='valid')(P_C2)
print(P2.shape)


(None, 140, 140, 128)


In [ ]:
P2_C1=tf.keras.layers.Conv2D(256,3,activation='relu')(P2)
print(P2_C1.shape)

(None, 138, 138, 256)


In [ ]:
P2_C2=tf.keras.layers.Conv2D(256,3,activation='relu')(P2_C1)
print(P2_C2.shape)

(None, 136, 136, 256)


In [ ]:
P3=tf.keras.layers.MaxPool2D(pool_size=(2,2),padding='valid')(P2_C2)
print(P3.shape)

(None, 68, 68, 256)


In [ ]:
P3_C1=tf.keras.layers.Conv2D(512,3,activation='relu')(P3)
print(P3_C1.shape)

(None, 66, 66, 512)


In [ ]:
P3_C2=tf.keras.layers.Conv2D(512,3,activation='relu')(P3_C1)
print(P3_C2.shape)

(None, 64, 64, 512)


In [ ]:
P4=tf.keras.layers.MaxPool2D(pool_size=(2,2),padding='valid')(P3_C2)
print(P4.shape)

(None, 32, 32, 512)


In [ ]:
P4_C1=tf.keras.layers.Conv2D(1024,3, activation='relu')(P4)
print(P4_C1.shape)

(None, 30, 30, 1024)


In [ ]:
P4_C2=tf.keras.layers.Conv2D(1024,3, activation='relu')(P4_C1)
print(P4_C2.shape)

(None, 28, 28, 1024)


In [ ]:
#https://www.kaggle.com/cjansen/u-net-in-keras
def get_crop_shape(target, refer):
        # width, the 3rd dimension
        cw = (target.get_shape()[2] - refer.get_shape()[2])
        assert (cw >= 0)
        if cw % 2 != 0:
            cw1, cw2 = int(cw/2), int(cw/2) + 1
        else:
            cw1, cw2 = int(cw/2), int(cw/2)
        # height, the 2nd dimension
        ch = (target.get_shape()[1] - refer.get_shape()[1])
        assert (ch >= 0)
        if ch % 2 != 0:
            ch1, ch2 = int(ch/2), int(ch/2) + 1
        else:
            ch1, ch2 = int(ch/2), int(ch/2)

        return (ch1, ch2), (cw1, cw2)

In [ ]:
Up_1 = tf.keras.layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same') (P4_C2)
print(Up_1.shape)

(None, 56, 56, 512)


In [ ]:
ch, cw = get_crop_shape(P3_C2, Up_1)

In [ ]:
ch, cw = get_crop_shape(P3_C2, Up_1)
crop_P3_C2 = tf.keras.layers.Cropping2D(cropping=(ch,cw))(P3_C2)
print(crop_P3_C2.shape)

(None, 56, 56, 512)


In [ ]:
ConCat1=tf.keras.layers.Concatenate()([Up_1,crop_P3_C2])
print(ConCat1.shape)

(None, 56, 56, 1024)


In [ ]:
Up1_C1=tf.keras.layers.Conv2D(512,3,activation='relu')(ConCat1)
print(Up1_C1.shape)

(None, 54, 54, 512)


In [ ]:
Up1_C2=tf.keras.layers.Conv2D(512,3,activation='relu')(Up1_C1)
print(Up1_C2.shape)

(None, 52, 52, 512)


In [ ]:
up_2=tf.keras.layers.Conv2DTranspose(256,(2,2),strides=(2,2),padding='same')(Up1_C2)
print(up_2.shape)

(None, 104, 104, 256)


In [ ]:
ch, cw = get_crop_shape(P2_C2, up_2)
crop_P2_C2 = tf.keras.layers.Cropping2D(cropping=(ch,cw))(P2_C2)
print(crop_P2_C2.shape)

(None, 104, 104, 256)


In [ ]:
ConCat2=tf.keras.layers.Concatenate()([up_2,crop_P2_C2])
print(ConCat2.shape)

(None, 104, 104, 512)


In [ ]:
Up2_c1=tf.keras.layers.Conv2D(256,3, activation='relu')(ConCat2)
print(Up2_c1.shape)


(None, 102, 102, 256)


In [ ]:
Up2_c2=tf.keras.layers.Conv2D(256,3, activation='relu')(Up2_c1)
print(Up2_c2.shape)

(None, 100, 100, 256)


In [ ]:
up_3=tf.keras.layers.Conv2DTranspose(128,kernel_size=(2,2),strides=(2,2),padding='same')(Up2_c2)
print(up_3.shape)

(None, 200, 200, 128)


In [ ]:
ch, cw = get_crop_shape(P_C2, up_3)
crop_P_C2 = tf.keras.layers.Cropping2D(cropping=(ch,cw))(P_C2)
print(crop_P_C2.shape)

(None, 200, 200, 128)


In [ ]:
ConCat3=tf.keras.layers.Concatenate()([up_3,crop_P_C2])
print(ConCat3.shape)

(None, 200, 200, 256)


In [ ]:
Up3_c1=tf.keras.layers.Conv2D(128,3,activation='relu')(ConCat3)
print(Up3_c1.shape)

(None, 198, 198, 128)


In [ ]:
Up3_c2=tf.keras.layers.Conv2D(128,3, activation='relu')(Up3_c1)
print(Up3_c2.shape)


(None, 196, 196, 128)


In [ ]:
up_4=tf.keras.layers.Conv2DTranspose(64,kernel_size=(2,2),strides=(2,2),padding='same')(Up3_c2)
print(up_4.shape)

(None, 392, 392, 64)


In [ ]:
ch, cw = get_crop_shape(C2, up_4)
crop_C2 = tf.keras.layers.Cropping2D(cropping=(ch,cw))(C2)
print(crop_C2.shape)


(None, 392, 392, 64)


In [ ]:
ConCat4=tf.keras.layers.Concatenate()([crop_C2,up_4])
print(ConCat4.shape)

(None, 392, 392, 128)


In [ ]:
Up4_c1=tf.keras.layers.Conv2D(64,3,activation='relu')(ConCat4)
print(Up4_c1.shape)

(None, 390, 390, 64)


In [ ]:
Up4_c2=tf.keras.layers.Conv2D(64,3,activation='relu')(Up4_c1)
print(Up4_c2.shape)

(None, 388, 388, 64)


In [ ]:
c_1x1=tf.keras.layers.Conv2D(1,1,activation='softmax',padding='same')(Up4_c2)
print(c_1x1.shape)

(None, 388, 388, 1)
